## Modelagem preditiva com dados Out-of-Core utilizando Dask

### Dados Out-of-Core 

Dados out-of-core são conjuntos de dados que ultrapassam a capacidade de memória RAM disponível em uma máquina. Este termo é frequentemente utilizado no contexto de bancos de dados, sistemas de arquivos e computação científica, onde o manuseio eficiente de grandes volumes de dados é crítico. 

O principal desafio ao trabalhar com dados out-of-core é o gerenciamento eficiente do I/O (entrada e saída) de dados, uma vez que o acesso contínuo a um disco rígido ou a uma rede pode se tornar um gargalo significativo. Além disso, é fundamental manter a eficiência no processamento para evitar a deterioração do desempenho devido ao constante carregamento e descarregamento de dados.

A capacidade de processar dados out-of-core é crucial para muitas aplicações de ciência de dados, especialmente em um mundo onde o volume de dados continua a crescer exponencialmente. As ferramentas que suportam esses processos são essenciais para permitir análises complexas e insights que de outra forma não seriam possíveis.

### Estratégias de Processamento

Para processar dados out-of-core de maneira eficaz, as estratégias a seguir são comumente adotadas:

1. **Streaming de dados**: Os dados são processados sequencialmente em pequenos blocos, permitindo que operações sejam realizadas em cada bloco à medida que passam pela memória.
2. **Indexação eficiente**: Criar índices que permitem acessar rapidamente partes específicas dos dados sem a necessidade de carregar conjuntos de dados inteiros na memória.
3. **Algoritmos otimizados para out-of-core**: Utilizar ou desenvolver algoritmos que minimizam a necessidade de movimentação de dados entre o disco e a RAM.
4. **Parallel processing**: Paralelizar o processamento de dados tanto quanto possível para maximizar o uso de todos os núcleos disponíveis, tratando diferentes partes dos dados simultaneamente.

### Ferramentas e Tecnologias

Várias ferramentas e tecnologias foram desenvolvidas para facilitar o manuseio de dados out-of-core:

- **Dask**: Fornece estruturas de dados e uma programação paralela que permite trabalhar eficientemente com grandes conjuntos de dados de maneira simples e integrada.
- **Pandas com chunksize**: Pandas pode processar arquivos grandes em pedaços menores com o parâmetro `chunksize`, permitindo operações em partes do arquivo de cada vez.
- **Bibliotecas de aprendizado de máquina adaptadas**: Algumas bibliotecas de machine learning, como o `scikit-learn`, têm opções ou versões que suportam aprendizado incrementativo para dados grandes.


O seguinte notebook tem como objetivo mostrar de forma detalhada como fazer um modelo preditivo no seu computador para dados que nem cabem na RAM faazendo uso do Dask. 

-------





# Introdução ao Dask

Dask é uma biblioteca de computação paralela e distribuída que integra de maneira harmoniosa com o ecossistema do Python, principalmente com as bibliotecas Pandas, NumPy e Scikit-Learn. Desenvolvido para operar com conjuntos de dados que excedem a memória RAM disponível, o Dask permite análises complexas e modelos de aprendizado de máquina em grandes volumes de dados.

O Dask tornou-se uma ferramenta essencial no arsenal de cientistas de dados que enfrentam desafios associados ao manejo de grandes conjuntos de dados. Sua habilidade em se integrar de forma suave ao ambiente Python, junto com sua simplicidade de uso e poderoso desempenho, o destaca como uma escolha primordial para computação de dados em grande escala.

### Por que usar Dask?

A crescente disponibilidade de grandes conjuntos de dados pode ser um desafio considerável para os cientistas de dados. A limitação da memória RAM significa que dados extensos não podem ser processados diretamente. A solução tradicional para isso seria usar ferramentas como Spark ou Hadoop, que, embora eficazes, podem ser complexas e pesadas para configurar e operar. O Dask, por outro lado, oferece uma interface simples e flexível, que reduz a complexidade e se integra facilmente ao fluxo de trabalho Python existente.

## Principais Características do Dask

- **Escalabilidade**: Dask é escalável verticalmente e horizontalmente, ou seja, pode processar dados em um único computador usando seus múltiplos núcleos ou em um cluster de máquinas.
- **Flexibilidade**: Suporta diversas operações como agregações, joins, leitura de dados em diferentes formatos, além de algoritmos de aprendizado de máquina.
- **Desempenho**: Utiliza computação paralela e otimizações inteligentes para acelerar a execução.
- **Lazy Execution**: O Dask adia a execução das operações até que seja explicitamente solicitado. Isso permite a otimização e paralelização eficientes dos cálculos.
- **Integração**: Trabalha de maneira nativa com as bibliotecas de ciência de dados Python, como Pandas, NumPy e Scikit-Learn, facilitando a adoção por parte de quem já está familiarizado com esses pacotes.

## Aplicações do Dask

O Dask é ideal para cenários onde o volume de dados é grande demais para a memória de uma máquina única, mas pequeno demais para justificar a complexidade de um sistema de processamento de dados distribuídos, como Hadoop ou Spark. É comumente utilizado em:

- Análises exploratórias de dados
- Processamento e limpeza de grandes conjuntos de dados
- Treinamento de modelos de machine learning em grandes volumes de dados
- Simulações e modelagens que exigem alta capacidade computacional


### Documentação

Para mais informações, acessar a documentação da biblioteca disponível em: https://docs.dask.org/en/stable/dataframe.html

-------


# Modelo Preditivo 

Os dados utilizados para o seguinte modelo são do Registro Nacional de Acidentes e Estatísticas de Trânsito (Renaest), sob a
coordenação do Departamento Nacional de Trânsito (Denatran),que organiza e junta os dados dos Detrans de
cada unidade federativa. Os dados desta base são alimentados pelos boletins de ocorrência da polícia.


## 1. Instalação e configuração 

Para instalar o Dask e outras bibliotecas necessárias, é necessário rodar o seguinte comando: 

```

!pip install dask[complete] 

```

In [1]:
import dask.dataframe as dd # import pandas as pd
import dask.array as da # import numpy as np
import matplotlib.pyplot as plt



In [3]:
acidentes = dd.read_csv('dados/Acidentes_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]
localidade = dd.read_csv('dados/Localidade_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]
veiculo = dd.read_csv('dados/TipoVeiculo_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]
vitimas = dd.read_csv('dados/Vitimas_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]

In [4]:
#merge dos dfs 
result = dd.merge(acidentes, localidade, on='chv_localidade', how='inner')
# result = dd.merge(result, veiculo, on='chv_localidade', how='inner')
# result = dd.merge(result, vitimas, on='chv_localidade', how='inner')
df_final = result.compute()


c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-pack

In [5]:
df_final.head(10)

,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,dia_semana,fase_dia,...,mes_referencia,mes_ano_referencia,regiao,uf,codigo_ibge_y,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante
0,352227.0,AC1200500201801,2018-01-16,AC,2018.0,1.0,12018.0,1200500.0,TERCA-FEIRA,MANHA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
1,28057.0,AC1200500201801,2018-01-06,AC,2018.0,1.0,12018.0,1200500.0,SABADO,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
2,1178931.0,AC1200500201801,2018-01-24,AC,2018.0,1.0,12018.0,1200500.0,QUARTA-FEIRA,MADRUGADA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
3,352225.0,AC1200500201801,2018-01-14,AC,2018.0,1.0,12018.0,1200500.0,DOMINGO,MANHA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
4,3515769.0,AC1200500201801,2018-01-04,AC,2018.0,1.0,12018.0,1200500.0,QUINTA-FEIRA,TARDE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
5,2860402.0,AC1200500201801,2018-01-06,AC,2018.0,1.0,12018.0,1200500.0,SABADO,MANHA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
6,2091362.0,AC1200500201801,2018-01-20,AC,2018.0,1.0,12018.0,1200500.0,SABADO,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
7,4524241.0,AC1200500201801,2018-01-08,AC,2018.0,1.0,12018.0,1200500.0,SEGUNDA-FEIRA,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
8,667583.0,AC1200500201801,2018-01-13,AC,2018.0,1.0,12018.0,1200500.0,SABADO,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
9,3254053.0,AC1200500201801,2018-01-14,AC,2018.0,1.0,12018.0,1200500.0,DOMINGO,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0


In [6]:
# Verificando a quantidade de valores faltantes por coluna
df_final.isnull().sum()

num_acidente                  0
chv_localidade                0
data_acidente                 0
uf_acidente                   0
ano_acidente                  0
mes_acidente                  0
mes_ano_acidente              0
codigo_ibge_x                 0
dia_semana                    0
fase_dia                      0
tp_acidente                   0
cond_meteorologica            0
end_acidente             156468
num_end_acidente              0
cep_acidente                  0
bairro_acidente         1926771
km_via_acidente               0
latitude_acidente       3624634
longitude_acidente      3624764
hora_acidente                 0
tp_rodovia                    0
cond_pista                    0
tp_cruzamento                 0
tp_pavimento                  0
tp_curva                      0
lim_velocidade                0
tp_pista                      0
ind_guardrail                 0
ind_cantcentral               0
ind_acostamento               0
qtde_acidente                 0
qtde_aci

In [7]:
# Filtrando apenas colunas numéricas
numeric_cols = df_final.select_dtypes(include=['float64', 'int64'])
# Calculando correlação
corr_matrix = numeric_cols.corr() 
corr_matrix

,num_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,num_end_acidente,cep_acidente,km_via_acidente,latitude_acidente,longitude_acidente,...,qtde_envolvidos,qtde_feridosilesos,qtde_obitos,ano_referencia,mes_referencia,mes_ano_referencia,codigo_ibge_y,qtde_habitantes,frota_total,frota_circulante
num_acidente,1.000000,0.001407,0.000472,0.000472,-0.000231,-0.000682,0.000312,0.000103,0.001483,0.000946,...,-0.000179,-0.000142,-0.000205,0.001407,0.000472,0.000472,-0.000231,-0.001220,-0.001018,-0.001054
ano_acidente,0.001407,1.000000,-0.105858,-0.105816,0.096092,0.093457,0.076875,0.023822,0.051282,0.037552,...,0.002572,0.007082,-0.025596,1.000000,-0.105858,-0.105816,0.096092,0.041655,0.076072,0.102727
mes_acidente,0.000472,-0.105858,1.000000,1.000000,0.010562,-0.010466,-0.000877,-0.010260,-0.006942,-0.005355,...,-0.025195,-0.025167,0.000666,-0.105858,1.000000,1.000000,0.010562,0.000261,0.004920,0.004821
mes_ano_acidente,0.000472,-0.105816,1.000000,1.000000,0.010566,-0.010462,-0.000874,-0.010259,-0.006941,-0.005354,...,-0.025195,-0.025167,0.000665,-0.105816,1.000000,1.000000,0.010566,0.000262,0.004923,0.004826
codigo_ibge_x,-0.000231,0.096092,0.010562,0.010566,1.000000,0.206205,-0.025245,-0.029688,0.111933,0.081969,...,0.025746,0.036113,-0.059484,0.096092,0.010562,0.010566,1.000000,0.019585,0.046882,0.065481
num_end_acidente,-0.000682,0.093457,-0.010466,-0.010462,0.206205,1.000000,-0.009300,0.000236,0.013642,0.009990,...,0.022383,0.025639,-0.019146,0.093457,-0.010466,-0.010462,0.206205,0.124352,0.115951,0.160849
cep_acidente,0.000312,0.076875,-0.000877,-0.000874,-0.025245,-0.009300,1.000000,-0.004265,0.013824,0.010124,...,0.016249,0.016842,-0.003884,0.076875,-0.000877,-0.000874,-0.025245,-0.000625,-0.016862,-0.006989
km_via_acidente,0.000103,0.023822,-0.010260,-0.010259,-0.029688,0.000236,-0.004265,1.000000,0.005484,0.004016,...,0.006720,0.003994,0.015198,0.023822,-0.010260,-0.010259,-0.029688,-0.008576,-0.007305,-0.005167
latitude_acidente,0.001483,0.051282,-0.006942,-0.006941,0.111933,0.013642,0.013824,0.005484,1.000000,0.741976,...,-0.013577,-0.011184,-0.013239,0.051282,-0.006942,-0.006941,0.111933,0.028405,0.028420,0.031399
longitude_acidente,0.000946,0.037552,-0.005355,-0.005354,0.081969,0.009990,0.010124,0.004016,0.741976,1.000000,...,-0.008974,-0.007371,-0.008870,0.037552,-0.005355,-0.005354,0.081969,0.018306,0.018938,0.020788


In [8]:
# Filtrando apenas as colunas com valor nulo
null_counts = df_final.isnull().sum()
print(null_counts[null_counts > 0])

end_acidente           156468
bairro_acidente       1926771
latitude_acidente     3624634
longitude_acidente    3624764
dtype: int64


In [9]:
df_final.head()

,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,dia_semana,fase_dia,...,mes_referencia,mes_ano_referencia,regiao,uf,codigo_ibge_y,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante
0,352227.0,AC1200500201801,2018-01-16,AC,2018.0,1.0,12018.0,1200500.0,TERCA-FEIRA,MANHA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
1,28057.0,AC1200500201801,2018-01-06,AC,2018.0,1.0,12018.0,1200500.0,SABADO,NOITE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
2,1178931.0,AC1200500201801,2018-01-24,AC,2018.0,1.0,12018.0,1200500.0,QUARTA-FEIRA,MADRUGADA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
3,352225.0,AC1200500201801,2018-01-14,AC,2018.0,1.0,12018.0,1200500.0,DOMINGO,MANHA,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0
4,3515769.0,AC1200500201801,2018-01-04,AC,2018.0,1.0,12018.0,1200500.0,QUINTA-FEIRA,TARDE,...,1.0,12018.0,NORTE,AC,1200500.0,SENA MADUREIRA,nao,43139.0,8100.0,6334.0


In [10]:
print(df_final.dtypes[:20])


num_acidente          float64
chv_localidade         object
data_acidente          object
uf_acidente            object
ano_acidente          float64
mes_acidente          float64
mes_ano_acidente      float64
codigo_ibge_x         float64
dia_semana             object
fase_dia               object
tp_acidente            object
cond_meteorologica     object
end_acidente           object
num_end_acidente      float64
cep_acidente          float64
bairro_acidente        object
km_via_acidente       float64
latitude_acidente     float64
longitude_acidente    float64
hora_acidente         float64
dtype: object


In [11]:
print(df_final.dtypes[20:40])

tp_rodovia               object
cond_pista               object
tp_cruzamento            object
tp_pavimento             object
tp_curva                 object
lim_velocidade           object
tp_pista                 object
ind_guardrail            object
ind_cantcentral          object
ind_acostamento          object
qtde_acidente           float64
qtde_acid_com_obitos    float64
qtde_envolvidos         float64
qtde_feridosilesos      float64
qtde_obitos             float64
ano_referencia          float64
mes_referencia          float64
mes_ano_referencia      float64
regiao                   object
uf                       object
dtype: object


In [12]:
print(df_final.dtypes[40:60])

codigo_ibge_y           float64
municipio                object
regiao_metropolitana     object
qtde_habitantes         float64
frota_total             float64
frota_circulante        float64
dtype: object


In [13]:
categorical_columns = [
    "chv_localidade",
    "data_acidente",
    "uf_acidente",
    "dia_semana",
    "fase_dia",
    "tp_acidente",
    "cond_meteorologica",
    "end_acidente",
    "bairro_acidente",
    "tp_rodovia",
    "cond_pista",
    "tp_cruzamento",
    "tp_pavimento",
    "tp_curva",
    "lim_velocidade",
    "tp_pista",
    "ind_guardrail",
    "ind_cantcentral",
    "ind_acostamento",
    "regiao",
    "uf",
    "municipio",
    "regiao_metropolitana"

]

numerical_columns = [  
    'latitude_acidente',
    'longitude_acidente',
    'num_acidente',
    'ano_acidente',
    'mes_acidente',
    'mes_ano_acidente',
    'codigo_ibge_x',
    'num_end_acidente',
    'cep_acidente',
    'km_via_acidente',
    'hora_acidente',
    'qtde_acidente',            
    'qtde_acid_com_obitos',     
    'qtde_envolvidos',          
    'qtde_feridosilesos',       
    'qtde_obitos',              
    'ano_referencia',           
    'mes_referencia',           
    'mes_ano_referencia',   
    'codigo_ibge_y',            
    'qtde_habitantes',          
    'frota_total',              
    'frota_circulante'

]

'''ordinal_columns = [
    'num_acidente',
    'ano_acidente',
    'mes_acidente',
    'mes_ano_acidente',
    'codigo_ibge_x',
    'num_end_acidente',
    'cep_acidente',
    'km_via_acidente',
    'hora_acidente',
    'qtde_acidente',            
    'qtde_acid_com_obitos',     
    'qtde_envolvidos',          
    'qtde_feridosilesos',       
    'qtde_obitos',              
    'ano_referencia',           
    'mes_referencia',           
    'mes_ano_referencia',   
    'codigo_ibge_y',            
    'qtde_habitantes',          
    'frota_total',              
    'frota_circulante'
                 
]'''

for column_group, column_type in (
    (categorical_columns, 'category'),
    (numerical_columns, 'float64'),
    #(ordinal_columns, 'int64'),
):
    for column in column_group:
        df_final[column] = df_final[column].astype(column_type)


# Split features and target, and return.
X = df_final.drop(columns=['num_acidente']).copy()
y = df_final['num_acidente'].copy()

## Visualização dos dados

## Separação treino-teste e modelagem inicial

In [14]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [16]:
categorical_features = X.select_dtypes(include=['category']).columns #variáveis categóricas
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns #variáveis numéricas

num_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('imputer', SimpleImputer(strategy='mean')),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(drop='first')),
])

preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_features),
        ('cat', cat_pipeline, categorical_features),
    ],
    remainder='passthrough',
)

pipe = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('regressor', DummyRegressor(strategy='mean')),
])

In [30]:
# from sklearn.model_selection import GridSearchCV, ShuffleSplit
# from sklearn.linear_model import Ridge
# from sklearn.linear_model import Lasso

import dask_ml
from dask_ml.model_selection import GridSearchCV , ShuffleSplit
from dask_ml.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from dask_ml.model_selection import train_test_split

# Quanto maior o numero de splits, maior a significância estatística da
# validação cruzada, mas também maior o tempo de execução.
num_splits = 100

param_grid = [
    {
        'regressor': [Lasso()],
        'regressor__alpha': [0.01, 0.1, 1, 10, 100, 500, 1000],
    },
]

test_fraction = 0.2
num_samples_total = len(y_train)
num_samples_test = int(test_fraction * num_samples_total)
num_samples_train = num_samples_total - num_samples_test

grid = GridSearchCV( #para encontrar os melhores parâmetros usando validação cruzada
    pipe,
    param_grid,
    cv=ShuffleSplit(
        n_splits=num_splits,
        test_size=num_samples_test,
        random_state=42,
    ),
    n_jobs=-1,
    scoring='neg_root_mean_squared_error',
)
grid.fit(X_train, y_train)

ValueError: Input contains NaN